# Environment Class Usage

Here we explore the different environment models available by default in RocketPy. Let's start by importing the rocketpy module.

In [ ]:
from rocketpy import Environment, SolidMotor, Rocket, Flight

If you are using Jupyter Notebooks, it is recommended to run the following line to make matplotlib plots which will be shown later interactive and higher quality.

In [ ]:
%matplotlib widget

## Datetime

RocketPy requires datetime information specifying year, month, day and hour to properly forecast the weather. Alongside this data, a timezone may also be specified as follows.

In [ ]:
env = Environment(date=(2022, 2, 16, 18), timeZone="America/New_York")

It is also possible to omit the timezone, in which case RocketPy will assume the datetime is given in standard UTC time, as done in the examples that follow. 

## Elevation

The elevation of the launch site of a rocket is particularly important for determining its interaction with the atmosphere and Earth's gravitational field.

Sometimes, together with the latitude and longitude of the launch site, the elevation which is measured in meters above sea level is known and can be used to initialize an Environment class instance as follows.

In [ ]:
env = Environment(
    date=(2019, 2, 10, 18),
    latitude=-21.960641,
    longitude=-47.482122,
    elevation=110,
)

However, this is not always the case. Lucky for us, there are alternatives to find an approximate value of the elevation of the launch site if the latitude and longitude values are know.

One option is to use the value supplied by some atmospheric models. Since elevation data is crucial for Numerical Weather Programing, some weather models make elevation data available together with other variables. But this will be covered later.

Another very useful and handy option is to use [Open-Elevation](https://open-elevation.com/), a free and open-source elevation API. It is built right into RocketPy and one can use it as follows.

First, we initialize a new Environment.

In [ ]:
env = Environment(date=(2019, 2, 10, 18), latitude=-21.960641, longitude=-47.482122)

Then, we use the Environment.setElevation method with the input parameter as 'Open-Elevation'. Note, however, that Open-Elevation servers are not always available.

In [ ]:
# env.set_elevation('Open-Elevation')

To get information from the Environment, just use the Environment.info() or the Environment.all_info() methods.

In [ ]:
env.info()

## Atmospheric Models

### Using the Standard Atmosphere

By default, when initializing an environment class, the International Standard Atmosphere as defined by ISO 2533 is initialized.

Note that the International Standard Atmosphere only has temperature and pressure profiles properly specified. Other profiles can be derived from it, however, winds are automatically set to 0 m/s.

In [ ]:
EnvISA = Environment()

In [ ]:
EnvISA.info()

The International Standard Atmosphere can also be reset at any time by using the following method.

In [ ]:
EnvISA.set_atmospheric_model(type="standard_atmosphere")

### Using a Custom Atmosphere

The user can also set a completely customized atmosphere. This is particularly useful for setting custom values of wind in both directions.

First, we initialize a new Environment.

In [ ]:
EnvCA = Environment()

Then, we set the atmospheric model as 'custom_atmosphere'.

Leaving the pressure field as None means we want the International Standard Atmosphere's pressure profile to be used. We could have done the same with temperature, but to showcase how floats can be used, we set the temperature field as a constant 300 K profile.

For the wind, we need to specify its value in both U (east) and V (north) direction. In this case, we used arrays to specify points. Consider a wind U profile of 5 m/s at 0 m and 10 m/s at 1000 m. For the wind V, we used -2 m/s at 0 m, 3 m/s at 500 m and 2 m/s at 1000 m.

In [ ]:
EnvCA.set_atmospheric_model(
    type="custom_atmosphere",
    pressure=None,
    temperature=300,
    wind_u=[(0, 5), (1000, 10)],
    wind_v=[(0, -2), (500, 3), (1600, 2)],
)

In [ ]:
EnvCA.all_info()

Keep in mind that importing .csv files for Custom Atmospheres is also possible. See the documentation on Environment.set_atmospheric_model for more information.

### Using Wyoming Upper Air Soundings

Another option to define atmospheric conditions is to use upper air soundings. The University of Wyoming - College of Engineering - Department of Atmospheric Sciences has a great compilation of atmospheric soundings on their website which can be found [here](http://weather.uwyo.edu/upperair/sounding.html).

For this example, we will use the sounding from 83779 SBMT Marte Civ Observations at 00Z 04 Feb 2019 which can be accessed using the URL generated by the above website: [http://weather.uwyo.edu/cgi-bin/sounding?region=samer&TYPE=TEXT%3ALIST&YEAR=2019&MONTH=02&FROM=0500&TO=0512&STNM=83779](http://weather.uwyo.edu/cgi-bin/sounding?region=samer&TYPE=TEXT%3ALIST&YEAR=2019&MONTH=02&FROM=0500&TO=0512&STNM=83779).

First, we initialize a new Environment. There is no need to specify dates, latitude and longitude here, so we will skip them for know.

In [ ]:
EnvWUAS = Environment()

Now, we set use the URL above as the input to our atmospheric model.

In [ ]:
URL = "http://weather.uwyo.edu/cgi-bin/sounding?region=samer&TYPE=TEXT%3ALIST&YEAR=2019&MONTH=02&FROM=0500&TO=0512&STNM=83779"

EnvWUAS.set_atmospheric_model(type="wyoming_sounding", file=URL)

Next, notice how even though we have not specified a surface elevation for our Environment, it has one set! The reason is that Wyoming Upper Air Soundings have elevation data for the sounding station, which is used to set the elevation of the Environment. If you do not want this elevation, you can change using Environment.setElevation(desired elevation in meters).

In [ ]:
EnvWUAS.all_info()

### Using NOAA's Ruc Soundings
Another option for upper air soundings is [NOAA's Ruc Soundings](https://rucsoundings.noaa.gov/) servers. It allows users to download virtual soundings from numerical weather prediction models such as GFS, RAP and NAM, and also allow for the download of real soundings from the Integrated Global Radiosonde Archive (IGRA).

All of these options can be retrieved as a text file which follows the GSD format. By generating such a file through the link above, the file's URL can be used to import the atmospheric data into RocketPy.

We will use the same sounding station as we did for Wyoming Soundings. Selecting ROABs as the initial data source, specifying the station through it's WMO-ID and opting for the ASCII (GSD format) button, the following page opens up: [https://rucsoundings.noaa.gov/get_raobs.cgi?data_source=RAOB&latest=latest&start_year=2019&start_month_name=Feb&start_mday=5&start_hour=12&start_min=0&n_hrs=1.0&fcst_len=shortest&airport=83779&text=Ascii%20text%20%28GSD%20format%29&hydrometeors=false&start=latest](https://rucsoundings.noaa.gov/get_raobs.cgi?data_source=RAOB&latest=latest&start_year=2019&start_month_name=Feb&start_mday=5&start_hour=12&start_min=0&n_hrs=1.0&fcst_len=shortest&airport=83779&text=Ascii%20text%20%28GSD%20format%29&hydrometeors=false&start=latest)

Let's initialize a new Environment to use this as it's source for atmospheric conditions. There is no need to specify dates, latitude and longitude here as well, so we will skip them for know.

In [ ]:
EnvRUC = Environment()

In [ ]:
URL = "https://rucsoundings.noaa.gov/get_raobs.cgi?data_source=RAOB&latest=latest&start_year=2019&start_month_name=Feb&start_mday=5&start_hour=12&start_min=0&n_hrs=1.0&fcst_len=shortest&airport=83779&text=Ascii%20text%20%28GSD%20format%29&hydrometeors=false&start=latest"

EnvRUC.set_atmospheric_model(type="NOAARucSounding", file=URL)

In [ ]:
EnvRUC.all_info()

### Using Weather Forecasts and Reanalysis

Weather Forecasts and Reanalysis can also be used to set the atmospheric model. Here we will showcase how to import global forecasts such as GFS and also local forecasts such as NAM and RAP for North America, all of these available through OPeNDAP in the [NOAA's NCEP NOMADS](http://nomads.ncep.noaa.gov/) website. Keep in mind that other generic forecasts can also be imported.

Furthermore, reanalysis such as ERA5, which can be easily downloaded from [ECMWF's Climate Data Store](https://cds.climate.copernicus.eu/#!/home), will also be showcased.

#### Setting up Date, Latitude and Longitude

All Environments which use weather forecasts or reanalysis must have date, latitude and longitude set up correctly during or after initialization in order for everything to work smoothly.

Let's first initialize a new Environment for the GFS weather forecast by specifying date, latitude and longitude during initialization. Keep in mind that date is specified using UTC time in year, month, day, UTC hour format.

In [ ]:
import datetime

tomorrow = datetime.date.today() + datetime.timedelta(days=1)

date_info = (tomorrow.year, tomorrow.month, tomorrow.day, 12)  # Hour given in UTC time

print("Tomorrow's date:", date_info)

In [ ]:
EnvGFS = Environment(date=date_info, latitude=-21.960641, longitude=-47.482122)

Another approach is to specify date, latitude and longitude values after initialization. Let's do this when creating a new Environment instance that will be using the NAM weather forecast.

In [ ]:
EnvNAM = Environment(rail_length=5)

In [ ]:
EnvNAM.set_date(date_info)

In [ ]:
EnvNAM.set_location(-21.960641, -47.482122)

#### GFS

Using the latest forecast from GFS is really simple. All we need to do is specify that GFS is the forecast we want. Note that since data is downloaded from NOMADS server, this line of code can take longer than usual.

In [ ]:
EnvGFS.set_atmospheric_model(type="Forecast", file="GFS")

In [ ]:
EnvGFS.all_info()

#### NAM and RAP

We can also request the latest forecasts from NAM and RAP. Since they are regional models for North America, we need to specify latitude and longitude points in North America. We will use [SpacePort America](https://www.google.com/maps/place/32°59'18.7%22N+106°58'30.2%22W/@32.9885323,-106.9772443,800m/data=!3m2!1e3!4b1!4m13!1m6!3m5!1s0x86df9591b4145bf9:0x19c787f1198f25cf!2sSpaceport+America!8m2!3d32.9903796!4d-106.975041!3m5!1s0x0:0x0!7e2!8m2!3d32.9885168!4d-106.9750625) for this, represented by coordinates 32.988528, -106.975056 or 32°59'18.7"N 106°58'30.2"W.

In [ ]:
EnvNAM = Environment(date=date_info, latitude=32.988517, longitude=-106.975062)

EnvRAP = Environment(date=date_info, latitude=32.988517, longitude=-106.975062)

In [ ]:
EnvNAM.set_atmospheric_model(type="Forecast", file="NAM")

EnvRAP.set_atmospheric_model(type="Forecast", file="RAP")

In [ ]:
EnvNAM.info()

In [ ]:
EnvRAP.info()

#### ERA-5 Reanalysis

Using data from ERA-5 reanalysis is as simple as specifying the file location and the dictionary to be used. This is much faster since the data has already been downloaded and is stored locally.

In [ ]:
EnvERA = Environment(
    date=(2018, 2, 17, 18),
    latitude=32.988517,
    longitude=-106.975062,
    elevation=1400,
)

In [ ]:
EnvERA.set_atmospheric_model(
    type="Reanalysis",
    file="../../data/weather/SpaceportAmerica_2018_ERA-5.nc",
    dictionary="ECMWF",
)

In [ ]:
EnvERA.info()

#### Other Weather Forecasts or Reanalysis - HIRESW

We can use other forecasts or reanalysis as well. As an example, we will use the [HIRESW CONUS](http://nomads.ncep.noaa.gov/txt_descriptions/HIRES_doc.shtml) forecast available at [NOAA's NCEP NOMADS](http://nomads.ncep.noaa.gov/) website.

In [ ]:
import datetime

today = datetime.date.today()
date_info = (today.year, today.month, today.day, 12)  # Hour given in UTC time
print("Today's date:", date_info)

EnvHIRESW = Environment(date=date_info, latitude=32.988517, longitude=-106.975062)

This model can be accessed through the following link: [https://nomads.ncep.noaa.gov:9090/dods/hiresw/hiresw20190206/hiresw_conusarw_12z](https://nomads.ncep.noaa.gov:9090/dods/hiresw/hiresw20190206/hiresw_conusarw_12z).

Since it is available from NOAA's NCEP NOMADS, we could use the NOAA dictionary However, we will create one from scratch to showcase the methodology.

The variables from the dataset can also be seen through the link above. We need to determine the name used by the model of the following dimensions and variables:

- Time
- Latitude
- Longitude
- Pressure Levels
- Geopotential Height (as a function of Time, Pressure Levels, Latitude and Longitude)
- Surface Geopotential Height (as a function of Time, Latitude and Longitude)
- Wind - U Component (as a function of Time, Pressure Levels, Latitude and Longitude)
- Wind - V Component (as a function of Time, Pressure Levels, Latitude and Longitude)

Looking through the variable list in the link above, we find the following correspondence:

- Time = **'time'**
- Latitude = **'lat'**
- Longitude = **'lon'**
- Pressure Levels = **'lev'**
- Geopotential Height = **'hgtprs'**
- Surface Geopotential Height = **'hgtsfc'**
- Wind - U Component = **'ugrdprs'**
- Wind - V Component = **'vgrdprs'**

Therefore, we use the dictionary shown below.

In [ ]:
HIRESW_dictionary = {
    "time": "time",
    "latitude": "lat",
    "longitude": "lon",
    "level": "lev",
    "temperature": "tmpprs",
    "surface_geopotential_height": "hgtsfc",
    "geopotential_height": "hgtprs",
    "u_wind": "ugrdprs",
    "v_wind": "vgrdprs",
}

This dictionary can then be used in the Environment.set_atmospheric_model method.

In [ ]:
date_string = f"{date_info[0]}{date_info[1]:02}{date_info[2]:02}"
print("Today's date string:", date_string)

EnvHIRESW.set_atmospheric_model(
    type="Forecast",
    file=f"https://nomads.ncep.noaa.gov/dods/hiresw/hiresw{date_string}/hiresw_conusarw_12z",
    dictionary=HIRESW_dictionary,
)

In [ ]:
EnvHIRESW.info()

### Using Windy Atmosphere

*Windy.com* is a website that lets you see and get weather and atmospheric forecasts in any place in the world.
The same atmospheric predictions and data that are available on *windy.com* can be set as an atmospheric model in RocketPy.

A few different models are accepted:

- *ECMWF-HRES*
- *GFS*
- *ICON-Global*
- *ICON-EU*.

#### Setting up Date, latitude and Longitude

Just like when using Forecasts and Reanalysis, all Environments which use windy must have date, latitude and longitude set up correctly during or after initialization in order for everything to work smoothly.

Lets first initialize a few different Envioronmets with the same date. Each one for a different atmospheric model.

In [ ]:
import datetime

tomorrow = datetime.date.today() + datetime.timedelta(days=1)

date_info = (tomorrow.year, tomorrow.month, tomorrow.day, 12)  # Hour given in UTC time

print("Tomorrow's date:", date_info)

In [ ]:
EnvECMWF = Environment(date=date_info, latitude=-21.960641, longitude=-47.482122)

EnvGFS = Environment(date=date_info, latitude=-21.960641, longitude=-47.482122)

EnvICON = Environment(date=date_info, latitude=-21.960641, longitude=-47.482122)

EnvICONEU = Environment(date=date_info, latitude=48.137154, longitude=11.576124)

#### Setting up the Atmospheric Models

Now we set up each different atmospheric model

In [ ]:
EnvECMWF.set_atmospheric_model(type="Windy", file="ECMWF")

EnvGFS.set_atmospheric_model(type="Windy", file="GFS")

EnvICON.set_atmospheric_model(type="Windy", file="ICON")

EnvICONEU.set_atmospheric_model(type="Windy", file="ICONEU")

And printing and plotting everything availabe (just for ECMWF model for simplicity)

In [ ]:
EnvICONEU.all_info()

### Using Ensemble Forecasts and Reanalysis

Ensemble weather forecasts and reanalysis can also be imported. Since ensembles are just a gathering of multiple weather predictions, they are great for dispersion analysis and Monte Carlo simulations. One member forecast of the ensemble can be selected to run each flight simulation.

Two global ensembles are available by default: GEFS and CMC Ensemble, but others can also be used. To showcase another example, an ERA5 Reanalysis ensemble will also be shown.

#### GEFS

In [ ]:
import datetime

tomorrow = datetime.date.today() + datetime.timedelta(days=1)
date_info = (tomorrow.year, tomorrow.month, tomorrow.day, 12)  # Hour given in UTC time

EnvGEFS = Environment(
    date=date_info,
    latitude=-21.960641,
    longitude=-47.482122,
    elevation=640,
)

In [ ]:
EnvGEFS.set_atmospheric_model(type="Ensemble", file="GEFS")

By default, the 0th forecast member is activated, which means that using EnvGFS will be exactly the same as using the 0th forecast member, and all other forecasts are kept away stored. Changing this is as easy as the following line of code.

In [ ]:
EnvGEFS.select_ensemble_member(1)

Now, a new forecast is active and EnvGFS behaves as if ensemble member 1 was its only forecast.

A very useful thing is to compare the forecast of all ensemble members, as well as confirming how many are there. All of this can be done by using Environment.all_info().

In [ ]:
EnvGEFS.all_info()

#### CMC Ensemble

In [ ]:
EnvCMC = Environment(
    date=date_info,
    latitude=-21.960641,
    longitude=-47.482122,
    elevation=640,
)

In [ ]:
EnvCMC.set_atmospheric_model(type="Ensemble", file="CMC")

In [ ]:
EnvCMC.all_info()

#### ERA5 Ensemble

In [ ]:
EnvERA5 = Environment(date=(2018, 2, 17, 18), latitude=-21.960641, longitude=-47.482122)

In [ ]:
EnvERA5.set_atmospheric_model(
    type="Ensemble",
    file="../data/weather/CuritibaRioSaoPauloEnsemble_2018_ERA-5.nc",
    dictionary="ECMWF",
)

In [ ]:
EnvERA5.all_info()